In [1]:
import numpy as np

In [2]:
def linear(X, W): #데이터와 가중치의 내적 함수
    H = np.dot(X, W)
    
    return H

In [3]:
def sigmoid(H): # 0과 1 사이의 값을 반납하는 sigmoid 함수
    p = 1 / (1 + np.exp(-H))
    
    return p

In [4]:
X = np.array([1, 2]) # 데이터

In [5]:
W = np.array([2, 3]) # 가충치

In [6]:
H = linear(X, W)
print(H) 
p = sigmoid(H)
print(p)
'''
sigmoid(H) = 1에 가까운 값
'''

8
0.9996646498695336


'\nsigmoid(H) = 1에 가까운 값\n'

In [7]:
W = np.array([-4, -3]) # 가중치

In [8]:
H = linear(X, W)
print(H)
p = sigmoid(H)
print(p)
'''
sigmoid(H) = 0에 가까운 값
'''

-10
4.5397868702434395e-05


'\nsigmoid(H) = 0에 가까운 값\n'

In [9]:
from sklearn.datasets import load_iris

In [10]:
data = load_iris()

In [11]:
x = data["data"]
y = data["target"]

In [12]:
print(x.shape)
x[:10]

(150, 4)


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

In [13]:
print(y.shape)
y

(150,)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [14]:
class Softmax:
    def __init__(self):
        
        self.params = {} # list initialization
        self.params['W'] = 0.0001 * np.random.randn(4, 3) # weight initialization (4,3)
        self.params['b'] = np.ones(3) # bias initialization (3)
    
    def forward(self, X):
        W = self.params['W']
        b = self.params['b']

        h = np.dot(X, W) + b # (3 ,1) hidden layer 만들기
        a = np.exp(h)
        #stable_a = np.exp(h - np.max(h, axis = 1).reshape(-1,1))
        p = a/np.sum(a, axis = 1).reshape(-1,1) # soft max 함수
        return p
    
    def loss(self, X, T):
        
        p = self.forward(X) #softmax 변환된 확률
        
        n = T.shape[0] # 데이터 수
        log_likelihood = 0 
        log_likelihood -= np.log(p[np.arange(n), T]).sum() # 실제 라벨에 대응되는 softmax 확률들의 log_likelyhood
        Loss = log_likelihood / n # n개의 loss값이 더해진 값을 n으로 나눠줌

        return Loss
    
    def accuracy(self, X, T):
        p = self.forward(X) #예측
        predict = np.argmax(p, axis = 1) #예측 결과 index 1darray 로 출력 
        
        return 1 - np.count_nonzero(predict - T)/len(T) #전체 값 분의 predict-T가 0인 값
        
    def gradient(self, X, T, learning_rate = 0.0001):
        
        p = self.forward(X) #softmax 변환된 확률
        
        t = np.zeros((T.shape[0], 3)) 
        t[np.arange(T.shape[0]), T] = 1
        #t는 인덱스 레이블 T를 One hot 벡터로 바꾼 것
        
        dp = p.copy() #list copy 미분값
        dp[np.arange(len(T)), T] -= 1 # loss 함수를 ai로 미분하면 pi - yi
        #목적함수에 대한 가중치 미분값을 담을 zero array 생성
        
        grads = {}
        grads['W'] = np.zeros((4, 3)) # weight initialization (4,3)
        grads['b'] = np.zeros(3) # bias initialization (3)
        #목적함수에 대한 가중치 미분값 합 구하기
        grads['W'] = (1/len(T)) * np.dot(X.T, p-t) #loss 함수를 w로 미분하면 dot(X.T, p-t)
        grads['b'] = (1/len(T)) * np.sum(p-t, axis = 0) #loss 함수를 b로 미분하면 p-t
        #p-t 대신 dp 사용 가능
        
        self.params['W'] -= learning_rate * grads['W'] # 기울기의 반대반향으로 W 조정
        self.params['b'] -= learning_rate * grads['b'] # 기울기의 반대반향으로 b 조정

In [18]:
softmax = Softmax()
# softmax class 호출

In [19]:
for i in range(5000):
    softmax.gradient(x, y)
    
    if i % 1000 == 0: 
        print(i, "번째 학습중입니다.")
        print("Accuracy : ", softmax.accuracy(x, y))
        print("Loss :     ", softmax.loss(x, y))
        print(softmax.params['W'])
        
#5000번 학습 1000번마다 출력

0 번째 학습중입니다.
Accuracy :  0.41333333333333333
Loss :      1.098325468410954
[[ 6.26061692e-05 -6.24096983e-05  1.03096840e-04]
 [ 1.41890272e-05 -1.30413724e-04 -1.72694373e-04]
 [-2.39860699e-04  1.44777533e-05  7.38216235e-05]
 [-5.03145348e-05  2.59013044e-05  5.27206251e-05]]
1000 번째 학습중입니다.
Accuracy :  0.33999999999999997
Loss :      1.0199954646749299
[[-0.00311427  0.00062693  0.00259063]
 [ 0.02405756 -0.01073946 -0.01360702]
 [-0.05802985  0.01471916  0.04315912]
 [-0.02546577  0.00354036  0.02195372]]
2000 번째 학습중입니다.
Accuracy :  0.6666666666666667
Loss :      0.9645286547874616
[[ 0.00870318 -0.00037711 -0.00822278]
 [ 0.05412849 -0.02186168 -0.03255572]
 [-0.10210576  0.02754727  0.07440693]
 [-0.04596722  0.00636237  0.03963315]]
3000 번째 학습중입니다.
Accuracy :  0.6666666666666667
Loss :      0.9162939939259718
[[ 0.02164534 -0.00160764 -0.01993441]
 [ 0.08324819 -0.03269643 -0.05084067]
 [-0.14203923  0.03937872  0.10250895]
 [-0.06470159  0.00876673  0.05596317]]
4000 번째 학습중입니다